In [0]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

Mount your Google Drive. In this notebook, we assume that 'report2' folder is placed directly under 'My Drive'.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!ls /content/drive/My\ Drive  #You should be able to see 'report2' folder by running this.

 00045.MTS
 00046.MTS
 20190611
 20190625
 20191220_mid_present.pdf
 answer20190514
'Colab Notebooks'
 day09
 DSC_0011.JPG
 handson20190514
 line_472323303084848.jpg
 line_472408253727160.jpg
 line_472430713069807.jpg
 line_7188681663453.jpg
 line_7201304176365.jpg
 line_7221473727451.jpg
 myresearch.mp4
 myresearch_out.mp4
 report1
 report2
 sample.mp4
 sample_test01.mp4
 sample_test02.mp4
 se2019_list.pdf
'Unity 2018.3.14f1 Personal - test01.unity - bending_exp1 - PC, Mac & Linux Standalone _DX11_ 2019-12-06 21-14-16.mp4'


Load the data.

In [14]:
root_dir="/content/drive/My Drive/report2/"
nsample = 3000

X_train = np.array(pd.read_csv(root_dir+"X_train.csv",header=None))
y_train = np.array(pd.read_csv(root_dir+"y_train.csv",header=None).ix[:,0])
X_test_tmp = np.array(pd.read_csv(root_dir+"X_test.csv",header=None))
print("x_train: ", X_train.shape)
print("y_train: ", y_train.shape)

XTrain_tmp = X_train[:nsample,:] #use the first 3000 samples for training
yTrain = y_train[:nsample]
XVal_tmp = X_train[nsample:,:] #use the rests for validation
yVal = y_train[nsample:]

print("var: ", np.var(X_train, axis=0))
del_list = np.where(np.var(X_train, axis=0) < 0.1)

print("index : ", del_list)
XTrain_before = np.delete(XTrain_tmp, del_list, axis=1)
XVal_before = np.delete(XVal_tmp, del_list, axis=1)
X_test_before = np.delete(X_test_tmp, del_list, axis=1)

print("XTrain:", XTrain_before.shape)

'''
XTrain_before = XTrain_tmp
XVal_before = XVal_tmp
X_test_before = X_test_tmp
'''
scaler = MinMaxScaler()
scaler.fit(XTrain_before)
XTrain = scaler.transform(XTrain_before)
XVal = scaler.transform(XVal_before)
X_test = scaler.transform(X_test_before)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """


x_train:  (5080, 561)
y_train:  (5080,)
var:  [0.00406658 0.00079975 0.00190986 0.20847212 0.25851543 0.1485247
 0.18561922 0.24176483 0.14594154 0.30391493 0.08172564 0.08190087
 0.1409845  0.11896527 0.07342377 0.21942734 0.06795175 0.01538646
 0.03103892 0.14255138 0.13874922 0.12780713 0.23110014 0.19755896
 0.13200301 0.10095428 0.0631926  0.06273898 0.05644977 0.06538651
 0.04584206 0.04338463 0.04868521 0.07854363 0.04456202 0.05554793
 0.05553283 0.11696298 0.10338543 0.13408047 0.2581824  0.15249291
 0.09543397 0.00540525 0.00551245 0.00708564 0.00516028 0.00536448
 0.00694707 0.25220316 0.14260049 0.09238177 0.2494125  0.1501379
 0.09730707 0.13696712 0.48379372 0.23448964 0.15852184 0.00448512
 0.00488681 0.00647788 0.12653444 0.07592381 0.15399803 0.04787449
 0.04600274 0.04613334 0.04817352 0.08975019 0.09378574 0.08977261
 0.08563902 0.0723725  0.06845945 0.06599748 0.06475627 0.48915349
 0.49062192 0.47936431 0.03419212 0.02896411 0.02295175 0.17844873
 0.19938688 0.0731

Train a linear SVM classifier

In [17]:
print("Training linear SVM classifier.") ##var<0.1 deleted and best_params: c=12.0, gamma=0.3
c_list = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0]
gamma_list = [0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
best_c = 0
best_score = 0
count = 0
for i in c_list:
  clf = svm.SVC(C = i, gamma = 0.3)
  clf.fit(XTrain,yTrain)
  yHatTrain = clf.predict(XTrain)
  yHatVal = clf.predict(XVal)
  print("iteration ", count)
  print("Training score ", len((np.where(yHatTrain == yTrain))[0])*1.0/XTrain.shape[0])
  print("Validation score ", len((np.where(yHatVal == yVal))[0])*1.0/XVal.shape[0])
  count = count + 1
  if best_score < len((np.where(yHatVal == yVal))[0])*1.0/XVal.shape[0]:
    best_score = len((np.where(yHatVal == yVal))[0])*1.0/XVal.shape[0]
    best_c = i
print("best_c", best_c)
print("best_score", best_score)
clf = svm.SVC(C = best_c, gamma = 0.3)
clf.fit(XTrain, yTrain)

Training linear SVM classifier.
iteration  0
Training score  0.9936666666666667
Validation score  0.9197115384615384
iteration  1
Training score  0.997
Validation score  0.926923076923077
iteration  2
Training score  0.998
Validation score  0.9264423076923077
iteration  3
Training score  0.999
Validation score  0.9259615384615385
iteration  4
Training score  0.999
Validation score  0.9288461538461539
iteration  5
Training score  0.999
Validation score  0.9298076923076923
iteration  6
Training score  0.9993333333333333
Validation score  0.9307692307692308
iteration  7
Training score  0.9993333333333333
Validation score  0.9298076923076923
iteration  8
Training score  1.0
Validation score  0.9302884615384616
iteration  9
Training score  1.0
Validation score  0.9307692307692308
iteration  10
Training score  1.0
Validation score  0.9302884615384616
iteration  11
Training score  1.0
Validation score  0.93125
iteration  12
Training score  1.0
Validation score  0.9317307692307693
iteration  1

SVC(C=13.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.3, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Evaluate training end validation scores.

In [0]:
yHatTrain = clf.predict(XTrain)
yHatVal = clf.predict(XVal)
print("Training score ", len((np.where(yHatTrain == yTrain))[0])*1.0/XTrain.shape[0])
print("Validation score ", len((np.where(yHatVal == yVal))[0])*1.0/XVal.shape[0])

Training score  0.9993333333333333
Validation score  0.9322115384615385


Predict test data and save the result.

In [0]:
yHatTest = clf.predict(X_test)
np.savetxt(root_dir+'result_svm.txt', yHatTest)